<h2>Load Data</h2>

In [2]:
import sys
sys.path += ['/work/pytorch_geometric', '/work/gaas/python']

In [4]:
import rmm

rmm.reinitialize(pool_allocator=True,initial_pool_size=5e+9, maximum_pool_size=20e+9)

In [5]:
import cudf
import cugraph
from cugraph.experimental import PropertyGraph
from cugraph.gnn.gaas_extensions import load_reddit
G = load_reddit(None, '/work/data/reddit')
G

In [6]:
from cugraph.gnn.pyg_extensions import CuGraphData
cd = CuGraphData(G, reserved_keys=['id','type'])

<h2>Training</h2>

In [7]:
import torch
from torch_geometric.data import Data

TRAINING_ARGS = {
    'batch_size':1000,
    'fanout':[10,25],
    'num_epochs':1,
}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

data = cd.to(device)
#data = Data(x=cd.x, edge_index = cd.edge_index, y = cd.y) #uncomment to run w/o cugraph

<h3>Create the Data Loader</h3>

In [8]:
from torch_geometric.loader import LinkNeighborLoader
import numpy as np

train_loader = LinkNeighborLoader(
    data,
    num_neighbors=TRAINING_ARGS['fanout'],
    batch_size=TRAINING_ARGS['batch_size'],
    shuffle=True
)

<h3>Define the GraphSAGE Model</h3>

In [9]:
import torch
from torch.nn import LSTM
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import GCNConv as SAGEConv

class SAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        for i in range(num_layers):
            in_channels = in_channels if i == 0 else hidden_channels
            out_channels = num_classes if i == num_layers - 1 else hidden_channels
            self.convs.append(SAGEConv(in_channels, out_channels))

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.3, training=self.training)
        return F.log_softmax(x, dim=1)



In [10]:
import pandas as pd
num_classes = len(pd.Series(np.array(cd.y, dtype=int)).unique())
num_classes

41

In [11]:
import torch.nn.functional as F
import tqdm
from math import ceil
from torch_geometric.loader import DataLoader
from torch.nn.functional import nll_loss

from datetime import datetime

import cupy
from cuml.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

#model = SAGE(data.num_node_features, hidden_channels=256, num_classes=num_classes, num_layers=3)
model = SAGE(data.num_node_features, hidden_channels=64, num_classes=num_classes, num_layers=1)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

num_batches = int(ceil(data.num_nodes / TRAINING_ARGS['batch_size']))
#num_batches = 5

def train():
    model.train()

    total_loss = 0
    for i, sampled_data in enumerate(tqdm.tqdm(train_loader)):
        if i == num_batches:
            return total_loss / data.num_nodes # FIXME is this right?
        sampled_data = sampled_data.to(device)
        print(f'iter: {i}')
        print('# nodes: ', sampled_data.num_nodes)
        print('# edges: ', sampled_data.num_edges)
        #out = model(sampled_data.x[-1], sampled_data.edge_index[0:2])
        out = model(sampled_data.x, sampled_data.edge_index)
        
        #loss = F.nll_loss(out, sampled_data.y[-1].T[0].to(torch.long))
        loss = F.nll_loss(out, sampled_data.y.T.to(torch.long))
        print(f'loss: {loss}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * out.size(0)

@torch.no_grad()
def encode(loader):
    model.eval()

    xs, ys = [], []
    for i, data in enumerate(loader):
        print(f'encode {i}')
        if i == num_batches:
            break

        out = model(data.x, data.edge_index)
        xs.append(torch.argmax(out, dim=1))
        ys.append(data.y)

    return torch.cat(xs, dim=0), torch.cat(ys, dim=0)

@torch.no_grad()
def test():
    model.eval()
    eval_loader = LinkNeighborLoader(data, num_neighbors=TRAINING_ARGS['fanout'], batch_size=TRAINING_ARGS['batch_size'])
    x_out, y_out = encode(eval_loader)

    val_acc = 0
    test_acc = balanced_accuracy_score(cupy.from_dlpack(y_out.__dlpack__()).get(), cupy.from_dlpack(x_out.__dlpack__()).get())

    return val_acc, test_acc, x_out, y_out


for epoch in range(1, 1 + TRAINING_ARGS['num_epochs']):
    train_start = datetime.now()
    loss = train()
    train_end = datetime.now()
    print('train time:', (train_end - train_start).total_seconds())

    test_start = datetime.now()
    val_acc, test_acc, x_out, y_out = test()
    test_end = datetime.now()
    print('test time:', (test_end - test_start).total_seconds())
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')

  0%|          | 0/57308 [00:00<?, ?it/s]

tensor([[219874, 187777, 228245,  ..., 121879, 145169,  47607],
        [113722,  25445, 120171,  ...,  51129, 138460,  25867]])


/tmp/ipykernel_895558/1844074848.py:36: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2318.)
  loss = F.nll_loss(out, sampled_data.y.T.to(torch.long))
  0%|          | 4/57308 [00:04<13:46:03,  1.16it/s]

iter: 0
# nodes:  97091
# edges:  450354
loss: 4.3689422607421875
tensor([[211697, 167350, 172156,  ..., 223217,  80141, 219378],
        [ 38240, 130106, 163294,  ..., 203705,  41801,  16440]])
iter: 1
# nodes:  97466
# edges:  448175
loss: 4.636002063751221
tensor([[215658, 190279, 163190,  ..., 165079, 201219, 185269],
        [122751,  11302,  74444,  ...,  61405,  86160, 111276]])
iter: 2
# nodes:  97540
# edges:  451118
loss: 4.749250888824463
tensor([[102052, 153952, 169805,  ..., 111486, 141571, 218766],
        [ 41351,  30117,  47180,  ...,  24727, 139923, 149910]])
iter: 3
# nodes:  96118
# edges:  451361
loss: 4.55747652053833
tensor([[ 28137,  68887, 193644,  ..., 188612, 225903, 135669],
        [ 21615,  13600, 108012,  ...,  43019, 149551, 123454]])
iter: 4
# nodes:  95841
# edges:  450055
loss: 4.037338733673096


  0%|          | 10/57308 [00:04<4:05:47,  3.89it/s]

...

tensor([[229532, 132834, 167500,  ..., 130935,  99995,  28689],
        [123353,  40084,   8754,  ...,  71024,  98093,  23350]])
iter: 223
# nodes:  95946
# edges:  448035
loss: 0.929855227470398
tensor([[105881,  69079, 133895,  ..., 217124, 223818, 167997],
        [ 75521,   3422,  59100,  ..., 189416, 163064,    407]])
iter: 224
# nodes:  95664
# edges:  451493
loss: 0.9186891913414001
tensor([[187468,  39390, 202846,  ..., 200141, 161077, 148887],
        [149592,    949,  92432,  ..., 172608,  65102,  79326]])
iter: 225
# nodes:  95290
# edges:  447404
loss: 0.9080442190170288
tensor([[139691, 195568, 171079,  ..., 110415, 205330, 188906],
        [ 72559, 141992,  32282,  ...,  78621, 178615,   9575]])
iter: 226
# nodes:  96167
# edges:  452465
loss: 0.9185481071472168
tensor([[220810, 224798, 166986,  ..., 202923, 182410, 133385],
        [ 73545,  13809, 162471,  ...,  47711,  44900,  45240]])
iter: 227
# nodes:  97151
# edges:  449947
loss: 0.9268539547920227


  0%|          | 232/57308 [00:14<39:44, 23.94it/s]

tensor([[187962, 168279, 198727,  ...,  65346,  42056, 192853],
        [ 40799,  21911, 150006,  ...,  52724,  27312, 166220]])
iter: 228
# nodes:  96734
# edges:  449679
loss: 0.9262142181396484
tensor([[ 87059, 168417,  87139,  ..., 208119, 226851, 175272],
        [ 41185,  22909,  48643,  ...,   1976,  10379, 161045]])
iter: 229
# nodes:  95806
# edges:  448857
loss: 0.9010607004165649
tensor([[226795,  54302, 174307,  ..., 218625, 223214, 118410],
        [ 19808,  17448, 119785,  ..., 166737, 142917, 104430]])
iter: 230
# nodes:  96534
# edges:  450286
loss: 0.9187257885932922
tensor([[103402, 149775, 145768,  ..., 181578, 174083, 109503],
        [ 26139,   4743, 139565,  ..., 171048,  39408, 104870]])
iter: 231
# nodes:  95477
# edges:  450606
loss: 0.9210151433944702
tensor([[216111, 198395, 208743,  ...,  75472, 108889, 178673],
        [143754, 178229, 140203,  ...,  19168,  71069, 123518]])
iter: 232
# nodes:  95861
# edges:  452378
loss: 0.9092644453048706
tensor([[134485

  0%|          | 233/57308 [00:15<1:02:09, 15.30it/s]


train time: 15.228701
tensor([[   242,    249,    524,  ..., 101446, 101864, 101878],
        [     0,      0,      0,  ...,      0,      0,      0]])
encode 0
tensor([[101975, 102023, 102098,  ..., 206575, 207161, 207246],
        [     0,      0,      0,  ...,      0,      0,      0]])
encode 1
tensor([[207267, 207403, 207404,  ..., 146822, 146920, 147601],
        [     0,      0,      0,  ...,      5,      5,      5]])
encode 2
tensor([[148463, 148466, 148764,  ...,  45025,  45209,  46031],
        [     5,      5,      5,  ...,      7,      7,      7]])
encode 3
tensor([[ 46740,  46909,  48515,  ..., 201906, 203685, 203841],
        [     7,      7,      7,  ...,     11,     11,     11]])
encode 4
tensor([[204196, 205619, 207021,  ..., 132979, 136185, 138400],
        [    11,     11,     11,  ...,     15,     15,     15]])
encode 5
tensor([[138670, 138671, 140582,  ...,   8606,   8652,   8889],
        [    15,     15,     15,  ...,     20,     20,     20]])
encode 6
tensor([[  8

In [ ]:
20.456 + 33.542

In [ ]:
test_acc

In [ ]:
s = None
for i,sampled_data in enumerate(train_loader):
    s = sampled_data
    break

In [ ]:
x_out = model(s.x, s.edge_index)

In [ ]:
x_out

In [ ]:
torch.argmax(x_out,dim=1)

In [ ]:
cd.num_node_features

In [ ]:
data.num_nodes / TRAINING_ARGS['batch_size']

In [ ]:
from torch_geometric.loader import DataLoader

batch = next(iter(DataLoader([data], batch_size=2)))

In [ ]:
for b in DataLoader([data],batch_size=16,follow_batch=['edge_index']):
    print(b)

In [ ]:
batch.edge_index.shape

In [ ]:
import cugraph
from cugraph import uniform_neighbor_sample
import cupy

sg = cugraph.Graph(directed=True)
sg.from_cudf_edgelist(
    G._edge_prop_dataframe.join(cudf.Series(cupy.ones(len(G._edge_prop_dataframe), dtype='float32'), name='weight')),
    source=PropertyGraph.src_col_name,
    destination=PropertyGraph.dst_col_name,
    edge_attr='weight',
    renumber=False
)

seeds = [1,2,3,4,5,6,9,100,234098]
td = cudf.Series()
ts = cudf.Series()
for n in [10,25]:
    r = uniform_neighbor_sample(
        sg,
        seeds,
        [n],
        True,
        False
    )

    indices = cupy.random.randint(0, len(r.destinations), n)
    seeds = r.destinations[indices]
    td = td.append(seeds)
    ts = ts.append(r.sources[indices])

In [ ]:
cugraph.subgraph(sg, cudf.concat([td, ts])).number_of_vertices()

In [ ]:
y_out

In [ ]:
from cugraph import uniform_neighbor_sample

uns = uniform_neighbor_sample(
        sg,
        [1],
        [10,25],
        True,
        False,
    )

In [ ]:
G = cugraph.Graph()
G.from_cudf_edgelist(uns, source='sources', destination='destinations', renumber=True)

In [ ]:
from scipy import sparse
el = G.edgelist.edgelist_df
ei = el.src.to_cupy(dtype='long')
ej = el.dst.to_cupy(dtype='long')
node = cudf.concat([el.src,el.dst]).unique()
es = len(node)
M = cupy.sparse.coo_matrix((cupy.ones_like(ei,dtype=float), (ei, ej)), shape=(es,es)).tocsc()
M.shape

In [ ]:
ej = el.src.astype('long')
ei = el.dst.astype('long')
node = torch.from_dlpack(cudf.concat([ei,ej]).unique().to_dlpack())
row = torch.from_dlpack(ei.to_dlpack())
col = torch.from_dlpack(ej.to_dlpack())
colptr = torch.ops.torch_sparse.ind2ptr(col, es)

In [ ]:
torch.stack([row,col],dim=0).shape

In [ ]:
node

In [ ]:
rdf = renumber_df
%timeit rdf[sampling_results.sources]

In [ ]:

rdf = renumber_df.sort_index()
rix = rdf.index.to_numpy()
v = rdf.sample(50).index.to_numpy()
%timeit rdf.iloc[np.searchsorted(rix,v)]

In [ ]:
rda = cupy.stack([nodes_of_interest.to_cupy(), cupy.arange(len(nodes_of_interest))], axis=-1)
rda = cupy.sort(rda,axis=0)
ixd = cupy.searchsorted(rda[:,0], sampling_results.destinations.to_cupy())
rda[ixd,1]

In [ ]:
sampling_results = uns
nodes_of_interest = cudf.concat([sampling_results.destinations, sampling_results.sources]).unique()
noi_tensor = torch.from_dlpack(nodes_of_interest.astype('long').to_dlpack())

renumber_df = cudf.Series(cudf.RangeIndex(len(nodes_of_interest)), index=nodes_of_interest)
src = torch.from_dlpack(renumber_df[sampling_results.sources].to_cupy().view(dtype='long').toDlpack())
dst = torch.from_dlpack(renumber_df[sampling_results.destinations].to_cupy().view(dtype='long').toDlpack())

print('asdf')


sampled_y = cd.y
if sampled_y is not None:
    sampled_y = sampled_y[noi_tensor]

print('bteg')

sampled_x = cd.x[noi_tensor]

data = Data(
    x=sampled_x, 
    edge_index=torch.concat([dst,src]).reshape((2,src.shape[0])),
    edge_attr=None,
    y=sampled_y
)

In [ ]:
uns.sources[uns.sources==1986]

In [ ]:
nodes_of_interest

In [ ]:
vertex_properties.index = new_vertex_id

In [ ]:
vertex_properties

In [ ]:
vertex_properties.sort_index()

In [ ]:
nodes_of_interest = uns.destinations.unique()
nodes_of_interest = cudf.concat([uns.destinations.unique(), uns.sources.unique()])

renumber_df = cudf.Series(cudf.RangeIndex(len(nodes_of_interest)), index=nodes_of_interest)

src = torch.from_dlpack(renumber_df[uns.sources].to_cupy().view(dtype='long').toDlpack())
dst = torch.from_dlpack(renumber_df[uns.destinations].to_cupy().view(dtype='long').toDlpack())

In [ ]:
data.neighbor_sample(torch.tensor([1,2,3,4]).cuda(), [10,25], True, True)

In [ ]:
nodes_of_interest = cupy.random.randint(0, 25000, 20000)
ns = data._CuGraphData__node_storage._data[data._CuGraphData__node_storage._feature_names]
%timeit sampled_x = torch.from_dlpack(cupy.take(ns.to_cupy(dtype='float32'), nodes_of_interest).toDlpack())

In [ ]:
nodes_of_interest = cupy.random.randint(0, 25000, 20000)
%timeit ns = data._CuGraphData__node_storage._data[data._CuGraphData__node_storage._feature_names]

In [ ]:
%timeit torch.from_dlpack(ns.iloc[nodes_of_interest].to_dlpack())

In [ ]:
%timeit sampled_x = torch.from_dlpack(data._CuGraphData__node_storage._data.iloc[nodes_of_interest, list(range(1,601))].to_dlpack())

In [ ]:
%timeit torch.from_dlpack(data._CuGraphData__node_storage._data.iloc[nodes_of_interest, list(range(1,601))].astype('float32').to_dlpack())

In [ ]:
%timeit torch.from_dlpack(data._CuGraphData__node_storage._data[data._CuGraphData__node_storage._feature_names].to_cupy(dtype='float32')[nodes_of_interest].toDlpack())

In [ ]:
from torch_geometric.cugraph.data.cugraph_storage import select

In [ ]:
def run(A):
    B = cupy.empty((len(nodes_of_interest), A.shape[1]), dtype='float32')
    I = nodes_of_interest.astype('int32')
    CuGraphData.select[16,256](A, B, I)
    return B

A = data._CuGraphData__node_storage._data[data._CuGraphData__node_storage._feature_names].to_cupy()
B = run(A)

In [ ]:
data._CuGraphData__node_storage._data.columns

In [ ]:
%timeit cupy.from_dlpack(data._CuGraphData__node_storage._data[data._CuGraphData__node_storage._feature_names].to_dlpack())

In [ ]:
data._CuGraphData__node_storage._CudfStorage__cached_keys

In [ ]:
a = np.array([
    [0,1],
    [1,2],
    [3,3],
    [5,4],
])

cupy.concatenate([cupy.array([0,1,2]), cupy.array([3,4,5])],axis=0,dtype='long').reshape((2,3))